# Final Data Report file for Seoul Neighborhood Analysis

* Build dataframe of neighborhoods in Seoul
* Get geo coordinates of each neighborhood
* Get data of venues in neighborhoods from Foursquare API
* Cluster neighborhoods using KNN to determine best neighborhood

## Import Libraries

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests 
!pip install beautifulsoup4
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print("Libraries imported.")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Libraries imported.


## Scrape data from Wikipedia

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Seoul").text

In [3]:
soup = BeautifulSoup(data, 'html.parser')

In [4]:
neighborhoods = []

In [5]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoods.append(row.text)

In [6]:
seoul = pd.DataFrame({"Neighborhood": neighborhoods})

seoul.head()

,Neighborhood
0,► Transport in Seoul by district‎ (23 C)
1,► Buildings and structures in Seoul by distri...
2,► Geography of Seoul by district‎ (25 C)
3,"► Dobong District‎ (3 C, 4 P)"
4,"► Dongdaemun District‎ (3 C, 6 P)"


Get rid of unnecessary rows

In [7]:
seoul = seoul.iloc[3:]
seoul.head()

,Neighborhood
3,"► Dobong District‎ (3 C, 4 P)"
4,"► Dongdaemun District‎ (3 C, 6 P)"
5,"► Dongjak District‎ (3 C, 4 P)"
6,"► Eunpyeong District‎ (2 C, 2 P)"
7,"► Gangbuk District‎ (3 C, 1 P)"


In [8]:
seoul.shape

(26, 1)

## Get geo coordinates

In [9]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [14]:
import geocoder

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Seoul, South Korea'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords


In [15]:
coords = [ get_latlng(neighborhood) for neighborhood in seoul["Neighborhood"].tolist() ]

In [17]:
coords

[[37.568260000000066, 126.97783000000004],
 [37.581890000000044, 127.05408000000011],
 [37.500560000000064, 126.95149000000004],
 [37.61846000000003, 126.92780000000005],
 [37.509089986158166, 127.01226885653118],
 [37.55039000000005, 127.14546000000007],
 [37.495100000000036, 127.06278000000009],
 [37.568260000000066, 126.97783000000004],
 [37.474860000000035, 126.89106000000004],
 [37.568260000000066, 126.97783000000004],
 [37.568260000000066, 126.97783000000004],
 [37.53913000000006, 127.08366000000001],
 [37.568260000000066, 126.97783000000004],
 [37.568260000000066, 126.97783000000004],
 [37.60199000000006, 127.1046100000001],
 [37.568260000000066, 126.97783000000004],
 [37.568260000000066, 126.97783000000004],
 [37.49056000000007, 127.0200000000001],
 [37.568260000000066, 126.97783000000004],
 [37.61505000000005, 127.02496000000008],
 [37.547840000000065, 127.02461000000005],
 [37.568260000000066, 126.97783000000004],
 [37.53532000000007, 126.84334000000001],
 [37.568260000000066

In [18]:
seoul_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
seoul_coords.head()

,Latitude,Longitude
0,37.56826,126.977830
1,37.58189,127.054080
2,37.50056,126.951490
3,37.61846,126.927800
4,37.50909,127.012269


In [25]:
seoul['Latitude'] = seoul_coords['Latitude']
seoul['Longitude'] = seoul_coords['Longitude']
seoul

,Neighborhood,Latitude,Longitude
3,"► Dobong District‎ (3 C, 4 P)",37.61846,126.927800
4,"► Dongdaemun District‎ (3 C, 6 P)",37.50909,127.012269
5,"► Dongjak District‎ (3 C, 4 P)",37.55039,127.145460
6,"► Eunpyeong District‎ (2 C, 2 P)",37.49510,127.062780
7,"► Gangbuk District‎ (3 C, 1 P)",37.56826,126.977830
8,"► Gangdong District‎ (3 C, 5 P)",37.47486,126.891060
9,"► Gangnam District‎ (3 C, 17 P)",37.56826,126.977830
10,"► Gangseo District, Seoul‎ (3 C, 2 P)",37.56826,126.977830
11,"► Geumcheon District‎ (3 C, 4 P)",37.53913,127.083660
12,"► Guro District, Seoul‎ (3 C, 8 P)",37.56826,126.977830


Drop rows with NaN values

In [32]:
seoul = seoul[seoul['Latitude'].notna()]
seoul

,Neighborhood,Latitude,Longitude
3,"► Dobong District‎ (3 C, 4 P)",37.61846,126.927800
4,"► Dongdaemun District‎ (3 C, 6 P)",37.50909,127.012269
5,"► Dongjak District‎ (3 C, 4 P)",37.55039,127.145460
6,"► Eunpyeong District‎ (2 C, 2 P)",37.49510,127.062780
7,"► Gangbuk District‎ (3 C, 1 P)",37.56826,126.977830
8,"► Gangdong District‎ (3 C, 5 P)",37.47486,126.891060
9,"► Gangnam District‎ (3 C, 17 P)",37.56826,126.977830
10,"► Gangseo District, Seoul‎ (3 C, 2 P)",37.56826,126.977830
11,"► Geumcheon District‎ (3 C, 4 P)",37.53913,127.083660
12,"► Guro District, Seoul‎ (3 C, 8 P)",37.56826,126.977830


Save as CSV file

In [33]:
seoul.to_csv("seoul.csv", index=False)

## Create map of Seoul with districts highlighted

In [34]:
# get the coordinates of Kuala Lumpur
address = 'Seoul, South Korea'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Seoul, South Korea is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Seoul, South Korea is 37.5666791, 126.9782914.


In [35]:
# create map of Seoul using latitude and longitude values
seoul_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(seoul['Latitude'], seoul['Longitude'], seoul['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(seoul_map) 
    
seoul_map

In [37]:
seoul_map.save('seoul_map.html')

## Use Foursquare API to explore neighborhoods

In [38]:
# define Foursquare Credentials and Version
CLIENT_ID = 'PK0UBGJ5HC3CZ2THOXRIZ3LFOCCA0LBTJUUM1Z01L0BDGTWG' # your Foursquare ID
CLIENT_SECRET = 'EB2XAL4ICB4CU4MFU2UQDIYYC2C45RSMVTNQYEXWFSO5VRMS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PK0UBGJ5HC3CZ2THOXRIZ3LFOCCA0LBTJUUM1Z01L0BDGTWG
CLIENT_SECRET:EB2XAL4ICB4CU4MFU2UQDIYYC2C45RSMVTNQYEXWFSO5VRMS


Get top 50 venues within radius of 2000 meters.

In [39]:
radius = 2000
LIMIT = 50

venues = []

for lat, long, neighborhood in zip(seoul['Latitude'], seoul['Longitude'], seoul['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [40]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1134, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Dobong District‎ (3 C, 4 P)",37.61846,126.9278,PENCIL5 Real Seoul hostel (펜슬5),37.616964,126.924293,Hostel
1,"► Dobong District‎ (3 C, 4 P)",37.61846,126.9278,Starbucks (스타벅스),37.611764,126.917264,Coffee Shop
2,"► Dobong District‎ (3 C, 4 P)",37.61846,126.9278,Drip & Dutch,37.613447,126.917626,Café
3,"► Dobong District‎ (3 C, 4 P)",37.61846,126.9278,알라딘 중고서점,37.617474,126.919960,Used Bookstore
4,"► Dobong District‎ (3 C, 4 P)",37.61846,126.9278,연신내문고,37.619952,126.920320,Bookstore


In [41]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"► Dobong District‎ (3 C, 4 P)",50,50,50,50,50,50
"► Dongdaemun District‎ (3 C, 6 P)",50,50,50,50,50,50
"► Dongjak District‎ (3 C, 4 P)",50,50,50,50,50,50
"► Eunpyeong District‎ (2 C, 2 P)",50,50,50,50,50,50
"► Gangbuk District‎ (3 C, 1 P)",50,50,50,50,50,50
"► Gangdong District‎ (3 C, 5 P)",50,50,50,50,50,50
"► Gangnam District‎ (3 C, 17 P)",50,50,50,50,50,50
"► Gangseo District, Seoul‎ (3 C, 2 P)",50,50,50,50,50,50
"► Geumcheon District‎ (3 C, 4 P)",50,50,50,50,50,50


Get number of unique categories

In [42]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 121 uniques categories.


In [43]:
venues_df['VenueCategory'].unique()[:50]

array(['Hostel', 'Coffee Shop', 'Café', 'Used Bookstore', 'Bookstore',
       'Park', 'Mountain', 'Chinese Restaurant', 'Korean Restaurant',
       'Trail', 'Supermarket', 'Bakery', 'Fast Food Restaurant',
       'Sushi Restaurant', 'Gym', 'Ice Cream Shop', 'Clothing Store',
       'Market', 'Bed & Breakfast', 'Department Store', 'Steakhouse',
       'Seafood Restaurant', 'Bunsik Restaurant', 'Multiplex', 'Buffet',
       'Electronics Store', 'Golf Course', 'Auto Workshop', 'BBQ Joint',
       'Dive Bar', 'Japanese Restaurant', 'Outlet Store',
       'Paper / Office Supplies Store', 'Deli / Bodega', 'Hotel',
       'Gym / Fitness Center', 'Flower Shop', 'Hotel Bar', 'Pie Shop',
       'Brazilian Restaurant', 'Shopping Mall', 'Cosmetics Shop',
       'Noodle House', 'Indoor Play Area', 'Italian Restaurant',
       'Performing Arts Venue', 'History Museum', 'Convenience Store',
       'Pub', 'Snack Place'], dtype=object)

## Neighborhood Analysis

In [45]:
# one hot encoding
seoul_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
seoul_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [seoul_onehot.columns[-1]] + list(seoul_onehot.columns[:-1])
seoul_onehot = seoul_onehot[fixed_columns]

print(seoul_onehot.shape)
seoul_onehot.head()

(1134, 122)


,Neighborhoods,Aquarium,Art Gallery,Art Museum,Asian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Big Box Store,Bistro,Bookstore,Bossam/Jokbal Restaurant,Brazilian Restaurant,Bridge,Bubble Tea Shop,Buffet,Bunsik Restaurant,Burger Joint,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Janguh Restaurant,Japanese Restaurant,Korean Restaurant,Lounge,Market,Metro Station,Mexican Restaurant,Mountain,Movie Theater,Multiplex,Museum,Noodle House,Other Great Outdoors,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pie Shop,Pizza Place,Plaza,Pub,Ramen Restaurant,Sake Bar,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store,Zoo
0,"► Dobong District‎ (3 C, 4 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"► Dobong District‎ (3 C, 4 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"► Dobong District‎ (3 C, 4 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"► Dobong District‎ (3 C, 4 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,"► Dobong District‎ (3 C, 4 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
seoul_grouped = seoul_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(seoul_grouped.shape)
seoul_grouped

(23, 122)


,Neighborhoods,Aquarium,Art Gallery,Art Museum,Asian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Big Box Store,Bistro,Bookstore,Bossam/Jokbal Restaurant,Brazilian Restaurant,Bridge,Bubble Tea Shop,Buffet,Bunsik Restaurant,Burger Joint,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Janguh Restaurant,Japanese Restaurant,Korean Restaurant,Lounge,Market,Metro Station,Mexican Restaurant,Mountain,Movie Theater,Multiplex,Museum,Noodle House,Other Great Outdoors,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pie Shop,Pizza Place,Plaza,Pub,Ramen Restaurant,Sake Bar,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store,Zoo
0,"► Dobong District‎ (3 C, 4 P)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.180000,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.000000,0.020000,0.020000,0.00,0.020000,0.000000,0.02,0.00,0.00,0.02,0.140000,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.000000,0.02,0.00,0.00,0.100000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.060000,0.00,0.00,0.00,0.00,0.00,0.00,0.100000,0.00,0.020000,0.000000,0.00,0.02,0.00,0.020000,0.00,0.00,0.00,0.00,0.00,0.00,0.020000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.00,0.00,0.020000,0.020000,0.00,0.02,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.02,0.00,0.00,0.00,0.000000,0.00,0.00,0.00
1,"► Dongdaemun District‎ (3 C, 6 P)",0.00,0.00,0.00,0.00,0.02,0.14,0.00,0.020000,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.02,0.00,0.000000,0.000000,0.000000,0.00,0.040000,0.000000,0.06,0.00,0.00,0.00,0.080000,0.00,0.02,0.02,0.02,0.00,0.00,0.04,0.000000,0.00,0.00,0.00,0.000000,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.02,0.02,0.020000,0.00,0.00,0.02,0.02,0.00,0.04,0.080000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.02,0.00,0.02,0.00,0.02,0.040000,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.020000,0.00,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00
2,"► Dongjak District‎ (3 C, 4 P)",0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.080000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.040000,0.000000,0.00,0.00,0.00,0.02,0.220000,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.060000,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.100000,0.00,0.00,0.00,0.02,0.00,0.04,0.080000,0.00,0.020000,0.000000,0.00,0.00,0.00,0.020000,0.00,0.02,0.00,0.00,0.00,0.00,0.040000,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.020000,0.00,0.02,0.00,0.00,0.000000,0.040000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00
3,"► Eunpyeong District‎ (2 C, 2 P)",0.02,0.00,0.00,0.00,0.00,0.08,0.00,0.120000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.000000,0.000000,0.02

In [48]:
len(seoul_grouped[seoul_grouped["Korean Restaurant"] > 0])

23

New dataframe for "Korean Restaurants"

In [50]:
seoul_res = seoul_grouped[["Neighborhoods","Korean Restaurant"]]
seoul_res.head()

,Neighborhoods,Korean Restaurant
0,"► Dobong District‎ (3 C, 4 P)",0.10
1,"► Dongdaemun District‎ (3 C, 6 P)",0.08
2,"► Dongjak District‎ (3 C, 4 P)",0.08
3,"► Eunpyeong District‎ (2 C, 2 P)",0.20
4,"► Gangbuk District‎ (3 C, 1 P)",0.10


## KNN Cluster

In [51]:
# set number of clusters
kclusters = 5

seoul_clustering = seoul_res.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seoul_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 1, 2, 3, 1, 3, 3, 3, 3], dtype=int32)

In [54]:
seoul_merged = seoul_res.copy()

seoul_merged["Cluster Labels"] = kmeans.labels_

In [55]:
seoul_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
seoul_merged.head()

,Neighborhood,Korean Restaurant,Cluster Labels
0,"► Dobong District‎ (3 C, 4 P)",0.10,3
1,"► Dongdaemun District‎ (3 C, 6 P)",0.08,1
2,"► Dongjak District‎ (3 C, 4 P)",0.08,1
3,"► Eunpyeong District‎ (2 C, 2 P)",0.20,2
4,"► Gangbuk District‎ (3 C, 1 P)",0.10,3


In [57]:
# merge seoul_grouped with seoul_data to add latitude/longitude for each neighborhood
seoul_merged = seoul_merged.join(seoul.set_index("Neighborhood"), on="Neighborhood")

print(seoul_merged.shape)
seoul_merged.head() # check the last columns!

(23, 5)


,Neighborhood,Korean Restaurant,Cluster Labels,Latitude,Longitude
0,"► Dobong District‎ (3 C, 4 P)",0.10,3,37.61846,126.927800
1,"► Dongdaemun District‎ (3 C, 6 P)",0.08,1,37.50909,127.012269
2,"► Dongjak District‎ (3 C, 4 P)",0.08,1,37.55039,127.145460
3,"► Eunpyeong District‎ (2 C, 2 P)",0.20,2,37.49510,127.062780
4,"► Gangbuk District‎ (3 C, 1 P)",0.10,3,37.56826,126.977830


In [58]:
print(seoul_merged.shape)
seoul_merged.sort_values(["Cluster Labels"], inplace=True)
seoul_merged

(23, 5)


,Neighborhood,Korean Restaurant,Cluster Labels,Latitude,Longitude
19,"► Seongbuk District‎ (4 C, 7 P)",0.140000,0,37.53532,126.843340
17,"► Seocho District‎ (3 C, 13 P)",0.120000,0,37.54784,127.024610
14,"► Jungnang District‎ (3 C, 3 P)",0.120000,0,37.49056,127.020000
21,"► Songpa District‎ (4 C, 9 P)",0.140000,0,37.53333,126.966670
11,"► Gwangjin District‎ (3 C, 8 P)",0.058824,1,37.60199,127.104610
1,"► Dongdaemun District‎ (3 C, 6 P)",0.080000,1,37.50909,127.012269
2,"► Dongjak District‎ (3 C, 4 P)",0.080000,1,37.55039,127.145460
5,"► Gangdong District‎ (3 C, 5 P)",0.080000,1,37.47486,126.891060
3,"► Eunpyeong District‎ (2 C, 2 P)",0.200000,2,37.49510,127.062780
20,"► Seongdong District‎ (3 C, 5 P)",0.100000,3,37.56826,126.977830


In [59]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(seoul_merged['Latitude'], seoul_merged['Longitude'], seoul_merged['Neighborhood'], seoul_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [60]:
map_clusters.save('map_clusters.html')

## Cluster Analysis

In [62]:
seoul_merged.loc[seoul_merged['Cluster Labels'] == 0]

,Neighborhood,Korean Restaurant,Cluster Labels,Latitude,Longitude
19,"► Seongbuk District‎ (4 C, 7 P)",0.14,0,37.53532,126.84334
17,"► Seocho District‎ (3 C, 13 P)",0.12,0,37.54784,127.02461
14,"► Jungnang District‎ (3 C, 3 P)",0.12,0,37.49056,127.02000
21,"► Songpa District‎ (4 C, 9 P)",0.14,0,37.53333,126.96667


In [63]:
#Cluster 1
seoul_merged.loc[seoul_merged['Cluster Labels'] == 1]

,Neighborhood,Korean Restaurant,Cluster Labels,Latitude,Longitude
11,"► Gwangjin District‎ (3 C, 8 P)",0.058824,1,37.60199,127.104610
1,"► Dongdaemun District‎ (3 C, 6 P)",0.080000,1,37.50909,127.012269
2,"► Dongjak District‎ (3 C, 4 P)",0.080000,1,37.55039,127.145460
5,"► Gangdong District‎ (3 C, 5 P)",0.080000,1,37.47486,126.891060


In [64]:
#Cluster 2
seoul_merged.loc[seoul_merged['Cluster Labels'] == 2]

,Neighborhood,Korean Restaurant,Cluster Labels,Latitude,Longitude
3,"► Eunpyeong District‎ (2 C, 2 P)",0.2,2,37.4951,127.06278
